In [1]:
import time
from datetime import datetime

end_time = datetime.now().replace(hour=11, minute=2, second=0)
if datetime.now() > end_time:
    end_time = end_time.replace(day=end_time.day + 1)

while datetime.now() < end_time:
    time.sleep(60)

In [3]:
from trading_helper import TradingHelper
import time
import statistics
import math
from datetime import datetime as dt
app=TradingHelper()
mail="cirigliano.santiago@gmail.com"
cantidad_minima=5
terms=["24hs","CI"]
maximum_vol=50
lista = ["MR36O", "LECHO"]
combinations = (lista, lista[::-1])  # Original y lista invertida
while dt.now().hour < 11: time.sleep(60)
while 11<= dt.now().hour < 17:
 if dt.now().hour >= 17:break
 for term in terms:
  term="CI"
  for position in (1,0):
    tickers=combinations[position]
    time.sleep(0.5)
    try: amount_in_portfolio=app.stocks_available(mail=mail,ticker=tickers[0],term=term,currency="pesos")
    except:amount_in_portfolio=app.stocks_available(mail=mail,ticker=tickers[0],term=term,currency="pesos")
    if amount_in_portfolio!=None and amount_in_portfolio != 0:
        amount_in_portfolio=amount_in_portfolio[term]
        try:
            numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
            denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
        except Exception as e:
            # Reintentar mientras el error sea un 500
            while "Error code: 500" in str(e):
                try:
                    numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
                    denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
                    break  # Salir del bucle si la solicitud es exitosa
                except Exception as e:
                    if "Error code: 500" not in str(e):
                        print(f"Error al obtener snapshot: {e}")  # Registro del error diferente
                        break  # Salir del bucle si es otro error
                    continue  # Continúa reintentando si es un error 500
        numerador =  [item for item in numerador if item['currency'] == 'ARS' and item['term'] == term][0]
        denominador =  [item for item in denominador if item['currency'] == 'ARS' and item['term'] == term][0]
        precio_venta_numerador=numerador['bids'][0]['price']#precio al que vendo este ticker
        vol_venta_numerador=numerador['bids'][0]['size']

        precio_compra_denominador=denominador['asks'][0]['price']#precio al que compro este ticker
        vol_compra_denominador=denominador['asks'][0]['size']
        print(f"{tickers[0]}, {term}, {amount_in_portfolio}")
        amount_in_portfolio=min(amount_in_portfolio, maximum_vol)
        minimum_vol = min(vol_compra_denominador,vol_venta_numerador,amount_in_portfolio)
        if precio_venta_numerador==0 or precio_compra_denominador==0 or minimum_vol == 0:break
        #tengo numerador y lo vendo para comprar denominador
        ratio=precio_venta_numerador/precio_compra_denominador
        min_ratio=1.00
        print(f"Ratio {ratio} vendiendo {vol_venta_numerador} de {tickers[0]} a precio venta {precio_venta_numerador} y comprando {vol_compra_denominador} de {tickers[1]} a precio venta {precio_compra_denominador}")
        if ratio>min_ratio:
            #Este bloque gestiona la cantidad de papeles a comprar y vender según dónde este la limitante del volumen
            #Maximiza la cantidad de recompra a 24 horas para hacer interes compuesto
            if minimum_vol == vol_venta_numerador:
                vol_compra_denominador = math.floor(vol_venta_numerador * (ratio))
            elif minimum_vol == amount_in_portfolio:
                vol_venta_numerador= amount_in_portfolio
                vol_compra_denominador = math.floor(vol_venta_numerador * (ratio))
            elif minimum_vol== vol_compra_denominador:
                vol_venta_numerador = math.floor(vol_compra_denominador /(ratio))
            print(f"Oportunidad ratio {ratio} vendiendo {vol_venta_numerador} de {tickers[0]} a precio venta {precio_venta_numerador} y comprando {vol_compra_denominador} de {tickers[1]} a precio venta {precio_compra_denominador}")
            sell_order=app.place_sell_order(mail=mail,
                                            ticker=tickers[0],
                                            quantity=vol_venta_numerador,
                                            price=precio_venta_numerador,
                                            term=term,
                                            currency="pesos")
            print(f"Orden de venta {tickers[0]} {sell_order}")
            order_status=app.check_order_status(mail=mail,
                                                order_number=sell_order["Orden"])
            print(order_status)
            start_time = time.time()
            while order_status == "MARKET":
                elapsed_time = time.time() - start_time
                if elapsed_time > 4:
                    cancel_order=app.cancel_order(mail=mail,order_number=sell_order["Orden"])
                    print(f"Orden cancelada por Timeout: {cancel_order}")
                    break
                order_status=app.check_order_status(mail=mail,
                                                order_number=sell_order["Orden"])
                print(f"Order Status de venta {order_status}")
                
            if order_status == "EXECUTED":
                buy_order=app.place_buy_order(mail=mail,
                                            ticker=tickers[1],
                                            quantity=vol_compra_denominador,
                                            price=precio_compra_denominador,
                                            term=term,
                                            currency="pesos")
                print(f"Orden de compra {tickers[1]} {buy_order}")
                order_status=app.check_order_status(mail=mail,
                                                    order_number=buy_order["Orden"])
                while order_status == "MARKET":
                    try:
                        time.sleep(1)
                        order_status=app.check_order_status(mail=mail, order_number=buy_order["Orden"])
                        if order_status =="FILLED": break
                        cancel_order=app.cancel_order(mail=mail,order_number=buy_order["Orden"])
                        precio_compra_denominador = precio_compra_denominador - 0.1
                        buy_order=app.place_buy_order(mail=mail,
                            ticker=tickers[1],
                            quantity=vol_compra_denominador,
                            price=precio_compra_denominador,
                            term=term,
                            currency="pesos")
                    except Exception as e: print(e)
                    
                print(f"Order status de Compra {order_status}")
                if buy_order["Success"] == True: print("Negocio ATR")

(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['LECHO', 'MR36O']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])
['MR36O', 'LECHO']
(['MR36O', 'LECHO'], ['LECHO', 'MR36O'])